![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🎯 Hands-On Exercise: Tool Selection Optimization

## Learning Objective (20-25 minutes)
Practice improving tool selection through hands-on exercises with real tool confusion scenarios.

## Prerequisites
- Completed `03_tool_selection_strategies.ipynb`
- Understanding of tool selection challenges
- Redis Stack running with course data
- OpenAI API key configured

---

## 🎯 Your Mission

Complete these practical exercises to master tool selection optimization:

1. **Improve a tool** with vague descriptions
2. **Test tool selection** with challenging queries
3. **Find confusion** between similar tools
4. **Consolidate tools** to reduce complexity

**Each exercise builds on the previous one!**

In [ ]:
# Setup - Run this first
import os
import asyncio
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# LangChain imports
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Redis and course management
import redis
from redis_context_course.course_manager import CourseManager

load_dotenv()
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
redis_client = redis.from_url(REDIS_URL)
course_manager = CourseManager()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Setup complete - ready for tool selection exercises!")

## Exercise 1: Improve a Tool Description

Take this tool with a vague description and rewrite it with clear guidance and examples.

In [ ]:
# Original tool with vague description
@tool
async def search_courses_vague(query: str) -> str:
    """Search for courses."""
    try:
        results = await course_manager.search_courses(query, limit=5)
        if not results:
            return "No courses found."
        output = []
        for course in results:
            output.append(f"{course.code}: {course.title}")
        return "\n".join(output)
    except Exception as e:
        return f"Error: {str(e)}"

print("❌ Original tool with vague description created")

In [ ]:
# TODO: Improve this tool's description
@tool
async def search_courses_improved(query: str) -> str:
    """
    # TODO: Write a much better description that includes:
    # - What this tool does specifically
    # - When the LLM should use it (with examples)
    # - What kind of queries work best
    # - What the output format will be
    # - When NOT to use it
    # 
    # Example structure:
    # "Search for courses using semantic similarity matching.
    # 
    # Use this when students ask about:
    # - Topics: 'machine learning courses', 'web development'
    # - Characteristics: 'beginner courses', 'online courses'
    # - General exploration: 'what courses are available?'
    # 
    # Do NOT use for:
    # - Specific course codes (use get_course_details instead)
    # - Prerequisites checking (use check_prerequisites instead)
    # 
    # Returns: List of up to 5 relevant courses with codes and titles."
    """
    
    # Same implementation
    try:
        results = await course_manager.search_courses(query, limit=5)
        if not results:
            return f"No courses found matching '{query}'. Try different keywords or broader terms."
        
        output = [f"Found {len(results)} courses matching '{query}':"]
        for course in results:
            output.append(f"• {course.code}: {course.title}")
        return "\n".join(output)
    except Exception as e:
        return f"Error searching courses: {str(e)}"

## Exercise 2: Test Tool Selection

Create 10 test queries and verify the LLM selects the right tool each time.

In [ ]:
# Create a simple agent to test tool selection
def create_test_agent(tools):
    """Create an agent with the given tools for testing."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful course advisor. Use the available tools to help students."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    agent = create_openai_functions_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

# TODO: Create additional tools for testing
@tool
async def get_course_details(course_code: str) -> str:
    """
    Get detailed information about a specific course by its code.
    
    Use this when:
    - Student asks about a specific course code ("Tell me about CS101")
    - Student wants detailed course information
    - Student asks about course description, prerequisites, or credits
    
    Do NOT use for:
    - Searching for courses by topic (use search_courses instead)
    - Finding multiple courses (use search_courses instead)
    
    Returns: Complete course details including description, prerequisites, and credits.
    """
    try:
        course = await course_manager.get_course_by_code(course_code.upper())
        if not course:
            return f"Course {course_code} not found. Please check the course code."
        
        details = f"**{course.code}: {course.title}**\n"
        details += f"Credits: {course.credits}\n"
        details += f"Description: {course.description}\n"
        if course.prerequisites:
            details += f"Prerequisites: {', '.join(course.prerequisites)}\n"
        return details
    except Exception as e:
        return f"Error getting course details: {str(e)}"

print("✅ Test tools created")

In [ ]:
# TODO: Create 10 test queries and predict which tool should be used
test_queries = [
    # TODO: Add test queries that should use search_courses_improved
    "What machine learning courses are available?",  # Should use: search_courses_improved
    "Show me programming courses",                    # Should use: search_courses_improved
    
    # TODO: Add test queries that should use get_course_details
    "Tell me about CS101",                           # Should use: get_course_details
    "What are the prerequisites for MATH201?",       # Should use: get_course_details
    
    # TODO: Add more challenging queries
    "I want to learn about databases",               # Should use: ?
    "What's CS301 about?",                          # Should use: ?
    "Find me some easy courses",                     # Should use: ?
    "How many credits is PHYS101?",                  # Should use: ?
    "What courses can I take online?",               # Should use: ?
    "Give me details on the intro programming course" # Should use: ?
]

# TODO: For each query, predict which tool should be used and why
# Then test with the agent to see if your predictions are correct

print(f"📝 Created {len(test_queries)} test queries")
print("\n🤔 Before testing, predict which tool should be used for each query!")

## Exercise 3: Find Confusion Between Similar Tools

Create two similar tools and test queries that could match either. How can you improve the descriptions?

In [ ]:
# TODO: Create two confusingly similar tools
@tool
async def list_courses(department: str) -> str:
    """List courses in a department."""
    # TODO: Implement this tool
    pass

@tool  
async def browse_courses(subject: str) -> str:
    """Browse courses by subject."""
    # TODO: Implement this tool
    pass

# TODO: Create test queries that could match either tool
confusing_queries = [
    "Show me computer science courses",
    "What courses are in the math department?",
    "I want to see physics courses"
]

# TODO: Test these queries and see which tool gets selected
# TODO: Improve the tool descriptions to eliminate confusion

print("❓ Created confusing tools - which one would you pick for each query?")

## Exercise 4: Consolidate Tools

If you have 5+ similar tools, try consolidating them into 1-2 flexible tools.

In [ ]:
# TODO: Imagine you have these 5 similar tools:
# - search_by_topic(topic)
# - search_by_department(dept) 
# - search_by_difficulty(level)
# - search_by_format(format)
# - search_by_instructor(name)
#
# How would you consolidate them into 1-2 tools?
# Consider:
# - Parameter design (required vs optional)
# - Tool naming and descriptions
# - User experience and clarity

class CourseSearchInput(BaseModel):
    """Input schema for comprehensive course search."""
    
    # TODO: Design parameters that can handle all the search types above
    # Hint: Think about what's required vs optional
    # Hint: Consider using Union types or enums for structured options
    
    query: str = Field(
        description="# TODO: Describe what goes in the main query parameter"
    )
    
    # TODO: Add optional filter parameters
    # department: Optional[str] = Field(default=None, description="...")
    # difficulty: Optional[str] = Field(default=None, description="...")
    # etc.

@tool(args_schema=CourseSearchInput)
async def search_courses_consolidated(query: str, **filters) -> str:
    """
    # TODO: Write a description for your consolidated tool
    # - Explain how it replaces multiple tools
    # - Give examples of different ways to use it
    # - Show how filters work
    """
    
    # TODO: Implement the consolidated search logic
    # This would combine all the search functionality
    pass

print("🔄 Design your consolidated tool to replace 5 separate tools!")

## Reflection Questions

After completing the exercises, think about:

**Tool Description Quality:**
- What made the improved descriptions better?
- How do examples help the LLM choose correctly?
- When is it helpful to specify what NOT to use a tool for?

**Tool Selection Testing:**
- Which queries were hardest for the LLM to handle?
- What patterns did you notice in successful vs failed selections?
- How can you make ambiguous queries clearer?

**Tool Consolidation:**
- When should you consolidate tools vs keep them separate?
- How do you balance flexibility with simplicity?
- What are the trade-offs of fewer, more complex tools?

**Next Steps:**
- How would you apply these lessons to your own agent?
- What tools in your project might be confusing?
- How could you test tool selection systematically?

---

## 🎉 Congratulations!

You've mastered tool selection optimization through:
- ✅ **Description improvement** with clear examples and guidance
- ✅ **Systematic testing** of tool selection behavior
- ✅ **Confusion identification** between similar tools
- ✅ **Tool consolidation** for better organization

These skills are essential for building reliable AI agents with many tools!

**Ready for more advanced topics?** Continue with the next section to learn about agent architectures and deployment patterns.